In [1]:
from array import array
import numpy as np
import random
import struct
from sklearn.metrics import confusion_matrix
import os
import cv2
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pickle

In [2]:
def read(fname_img, fname_lbl):
    f = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", f.read(8))
    lbl = array("b", f.read())
    f.close()

    f = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", f.read(16))
    img = array("B", f.read())
    f.close()

    tmp = []
    cur_img = []
    img_matrix = []
    for x in img.tolist():
        tmp.append(x)
        if len(tmp) == 28:
            cur_img.append(tmp)
            tmp = []
        if len(cur_img) == 28:
            img_matrix.append(cur_img)
            cur_img = []

    return lbl.tolist(), img_matrix

In [20]:
def cal_sift(in_img):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(in_img, None)
    return kp, des

In [30]:
def sift_sample(imgs, sample_num):
    key_points = np.array([])
    descriptors = np.array([])
    index = []
    for i in range(sample_num):
        kp, des = cal_sift(np.array(imgs[i]))
        if (des == None):
            continue
        if len(key_points):
            key_points = np.append(key_points, kp)
            descriptors = np.append(descriptors, des, axis=0)
            index = index + [i] * len(kp)
        else:
            key_points = kp
            descriptors = des
            index = [i] * len(kp)
        if (i+1) % 100 == 0:
            print i+1, "sampled"
    return descriptors

In [5]:
def descriptor_cluster(descriptors, cluster_num, iter_nume):
    cluster = KMeans(n_clusters=cluster_num, max_iter=iter_num).fit(descriptors)
    return cluster

In [34]:
def img_to_feature(img, cluster, cluster_num):
    sift = cv2.xfeatures2d.SIFT_create()
    feature = [0] * cluster_num
    kp, des = cal_sift(img)
    if des == None:
        return np.array(feature)
    cluster_result = cluster.predict(des)
    for c in cluster_result:
        feature[c] += 1
    return np.array(feature)

In [7]:
def generate_features(imgs, cluster, cluster_num):
    features = []
    for img in imgs:
        f = img_to_feature(img, cluster, cluster_num)
        features.append(f)
    return np.array(features)

In [46]:
def train_and_test(features_train, features_test, lbl_train, lbl_test):
    clf = RandomForestClassifier(n_estimators=100, max_depth=20)
    clf.fit(features_train, lbl_train)
    result = clf.predict(features_test)
    error = sum([int(result[i] != test_lbl[i]) for i in range(len(result))])
    print "Accuracy: ", 1 - float(error)/len(result)
    print confusion_matrix(result, test_lbl)
    return clf

In [9]:
train_size = 60000

train_lbl, train_img = read("train-images-idx3-ubyte", "train-labels-idx1-ubyte")
test_lbl, test_img = read("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte")

index_shuf = range(train_size)
random.shuffle(index_shuf)
x = []
y = []
for i in range(train_size):
    x.append(train_img[i])
    y.append(train_lbl[i])
train_lbl, train_img = y, x

In [32]:
cluster_num = 200
sample_num = 10000
iter_num = 300

train_img = np.uint8(np.array(train_img))
descriptors = sift_sample(train_img, sample_num)
print descriptors.shape
cluster = descriptor_cluster(descriptors, cluster_num, iter_num)

100 sampled
200 sampled
300 sampled
400 sampled
500 sampled
600 sampled
700 sampled
800 sampled
900 sampled
1000 sampled
1100 sampled
1200 sampled
1300 sampled
1400 sampled
1500 sampled
1600 sampled
1700 sampled
1800 sampled
1900 sampled
2000 sampled
2100 sampled
2200 sampled
2300 sampled
2400 sampled
2500 sampled
2600 sampled
2700 sampled
2800 sampled
2900 sampled
3000 sampled
3100 sampled
3200 sampled
3300 sampled
3400 sampled
3500 sampled
3600 sampled
3700 sampled
3800 sampled
3900 sampled
4000 sampled
4100 sampled
4200 sampled
4300 sampled
4400 sampled
4500 sampled
4600 sampled
4700 sampled
4800 sampled
4900 sampled
5000 sampled
5100 sampled
5200 sampled
5300 sampled
5400 sampled
5500 sampled
5600 sampled
5700 sampled
5800 sampled
5900 sampled
6000 sampled
6100 sampled
6200 sampled
6300 sampled
6400 sampled
6500 sampled
6600 sampled
6700 sampled
6800 sampled
6900 sampled
7000 sampled
7100 sampled
7200 sampled
7300 sampled
7400 sampled
7500 sampled
7600 sampled
7700 sampled
7800 sam

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [36]:
features_train = generate_features(train_img, cluster, cluster_num)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [37]:
test_img = np.uint8(np.array(test_img))
features_test = generate_features(test_img, cluster, cluster_num)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [47]:
train_and_test(features_train, features_test, train_lbl, test_lbl)

Accuracy:  0.7107
[[844   1  50  14   6  66  78  20  16  30]
 [  9 940  40   8  22  42  84 141  17  29]
 [  5   4 627  50  27  25  19  84  19  10]
 [  4   1  21 765   7  14   3   9  27   6]
 [  2   1  13  21 768  17   2  18  25  28]
 [  2   0   5  61  11 485  33   3  10  25]
 [ 85 183 102  52  47 136 522  98  64  71]
 [ 13   3 144  23  27  56  39 622   6   9]
 [  4   1  17  11  23  12  11  11 747  14]
 [ 12   1  13   5  44  39 167  22  43 787]]


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)